**YOUR NAME**

Spring 2024

CS 251: Data Analysis and Visualization

# Lab 4c | PCA and Image Compression

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


plt.style.use(['seaborn-v0_8-colorblind', 'seaborn-v0_8-darkgrid'])
plt.rcParams.update({'font.size': 10})
plt.rcParams.update({'figure.figsize': [7,7]})

np.set_printoptions(suppress=True, precision=5)

## Task 1: Preparing the Image

### 1a. Import and view an image

1. Import `lorikeet.png` using pyplot's [imread](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imread.html)
2. Print the shape of the image. *NOTE: The 4th color channel corresponds to the image's [alpha channel](https://en.wikipedia.org/wiki/Alpha_channel)*.
3. Display the image using pyplot's [imshow](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html). Remove the grid lines using [plt.grid](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.grid.html).
4. Show the pyplot


### 1b. Reshape and compute original variable means

1. Reshape your image for compression
    1. Reshape your 3 dimensional `(R, C, n_chans)` array to be a 2 dimensional `(R, C*n_chans)` array and assign it to a new variable (`color_flat`).
    2. Print out the shape of `color_flat`.
2. Calculate the [means](https://numpy.org/doc/stable/reference/generated/numpy.mean.html) of each variable in  `color_flat` (columns) and assign it to a new variable. Your means should have the shape `(C*n_chans,)`

## Task 2:  Compressing the Image

### Find the Image's Principal Components

Use Sci-kit Learn's [PCA class](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to extract the image's eigenvectors and the principal components
1. Create an instance of the PCA class with 400 components
2. Call the method `fit_transform` with `color_flat` as the X parameter and saving the results into a new variable representing the Transformed values.
3. Print the shape of the Transformed values and the PCA's components.  The Transformed values should have the shape `(R, 400)` and the PCA's components should have the shape `(400, C*n_chans)`
4. Create an elbow plot of the [cumulative sum](https://numpy.org/doc/stable/reference/generated/numpy.cumsum.html) of the [PCA's](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) explained variance ratio.
    1. The x axis should be `# of Principal Components`
    1. The y axis should be `Proportional Variance Accounted for`
    1. Include a title, and x and y axis labels.

In [ ]:
from sklearn.decomposition import PCA



## Task 3: Reconstruct the Image 

### Reconstruct the image accounting for 90-91% percent of the proportional variance

1. Using the elbow plot, determine a number of principal components which account for roughly 90-91% percent of the proportional variance in the image.
1. Compute the inverse transform:
    1. `transformed_values @ principal_components + orig_var_means`
    2. With shapes `(R, k) @ (k, C*n_chans) + (C*n_chans,)`
    3. With `k` equal to the number of principal components you are keeping
3. Show the reconstructed image
    1. Use numpy's [clip](https://numpy.org/doc/stable/reference/generated/numpy.clip.html) method to clip any values less than 0 or greater than 1 in the reconstructed image.
    2. Reshape the reconstructed image back to the shape of the original image
    3. Display the image using pyplot's [imshow](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html)
    4. Show the pyplot
2. Compute the compression ratio of this image
    1. Uncompressed size =  the number of floats in the original image `R*C*n_chans`
    2. Compressed size = the number of floats required to compute the inverse transform `R*k + k*C*n_chans + C*n_chans`
    3. Compression ratio = Compressed size / Uncompressed size

## Turn in your lab

Follow the usual submission format and submit your lab on Google Classroom.